# Metadata analysis of articles with low scores marked by Tagger
This script takes the text file consisting PMIDs of the artices with low scores given by Tagger as an input and returns the text file with the metadata of the article. This metadata consists of Article PMID, Article title and Article Publication Type from the PubMed database. Analysis of these articles shows that the tool does not work as expected when the abstract of the article is absent.

In [ ]:
from xlrd import open_workbook
import urllib
from Bio import Entrez
from xml.dom import minidom
import xml.etree.ElementTree as ET

Entrez.email = "kansara3@illinois.edu"

# Read list of PMIDs from txt file
PMID_list_file = open("D:/GRA/rct_lessthan.txt","r")

Result_file = open("D:/GRA/rct_lessthan_Results.txt","w")

for PMID in PMID_list_file.readlines():
    PMID = PMID.replace("\n","")
    print(PMID)

    # Pass PMIDs to PubMed API to get articles' metadata in XML format
    link = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + PMID + "&retmode=xml&WebEnv=%3Cwebenv%20string%3E&usehistory=y"
    print(link)
        
    f = urllib.request.urlopen(link)
    
    
    tree = ET.parse(f)
    doc = tree.getroot()    
    
    # Get the metadata from the PubMed API
    for child in doc:
        for step_child in child:
            if step_child.tag == "MedlineCitation":
                for step_child_level_2 in step_child:
                    if step_child_level_2.tag == "PMID":
                        article_PMID =  step_child_level_2.text
                    if step_child_level_2.tag == "Article":
                        for step_child_level_3 in step_child_level_2:
                            if step_child_level_3.tag == "ArticleTitle":
                                article_title = step_child_level_3.text
                            
                            # Check whether Abstract is present in the metadata    
                            if step_child_level_3.tag == "Abstract":
                                article_abstract_list = []
                                for step_child_level_4 in step_child_level_3:
                                    if step_child_level_4.tag == "AbstractText":
                                        article_abstract_list.append(step_child_level_4.text)
                            else:
                                article_abstract_list = []
                            
                            # Check for the Publication type of the article in PubMed database
                            if step_child_level_3.tag == "PublicationTypeList":
                                article_publication_type_list = []
                                for step_child_level_4 in step_child_level_3:
                                    article_publication_type_list.append(step_child_level_4.text)
                           
                                print(article_title)
                                article_title = article_title.replace(",","")
                                article_abstract = " ".join(str(x) for x in article_abstract_list)   
                                article_publication_type = " ".join(str(x) for x in article_publication_type_list)   
                                article_abstract = article_abstract.replace(",","")
                                print(article_publication_type)
                                
                                article_string = article_PMID + "," + article_title + "," + article_publication_type + "\n"
                                # Write the result into output file
                                Result_file.write(article_string)
                                
Result_file.close()